# arbitrage-finder

In this notebook, I am investigating the implementation of the arbitrage finder. The idea is to get the current odd matrices from the `dashboardScraper`, then for each matrix to investigate if there are arbitrage opportunities

### Example
This example is taken from http://epchan.blogspot.de/2018/01/a-novel-capital-booster-sports-arbitrage.html

Suppose that the odd matrix is given by:
    
$$
A = 
\begin{bmatrix}
1.4 & 1.2 & 1.2 \\
8.8 & 9.5 & 9.1\\
5.8 & 6.0 & 6.8\\
\end{bmatrix}
$$
and let $x_1$, $x_2$ and $x_3$ be the ammount we allocate on the outcomes of the first and second row, following http://epchan.blogspot.de/2018/01/a-novel-capital-booster-sports-arbitrage.html, there exists an arbotrage opportunity by betting $x_1$ on $1.4$, $x_2$ on $9.5$ and $x_3$ on 6.8. 

An arbitrage opportunity arrises, if the sum of the inverse maximum of the odds along each row is less then 1, more precisely, if

$$
\sum_{i = 1}^n \frac{1}{\max_{j \in J}d_{ij}} - 1 < 0
$$

In that case, the profit from the arbitrage is given by

$$
1 - \sum_{i = 1}^n \frac{1}{\max_{j \in J}d_{ij}}
$$

and the available capital has to be allocated along each bet with proportion:

$$
w_i = \frac{P'_i}{\sum_{i = 1}^n P'_i} \qquad \text{where} \qquad P'_i = \frac{1}{|J|}\sum_{j \in J} \frac{1}{d_{ij}}
$$

In the example above $1 - (1/1.4 + 1/9.5 + 1/6.8) = 0.0333$, hence we can expect a profit of $3.34\%$.

We can extimate the probability of an event to occure, by the inverse of the corresponed odds:
$$
P'_1 = \frac{1}{1.4} = 0.714 \\
P'_X = \frac{1}{9.5} = 0.105 \\
P'_2 = \frac{1}{6.8} = 0.147 \\
$$
and the amounth of capital, which has to be allocated on each bet is given by
$$
w_1 = \frac{P'_1}{P'_1 + P'_X + P'_2} = 0.739 \\
w_X = \frac{P'_X}{P'_1 + P'_X + P'_2} = 0.109 \\
w_2 = \frac{P'_2}{P'_1 + P'_X + P'_2} = 0.152
$$


In [94]:
### imports
import pandas as pd
import sys

sys.path.append('..')
from src.dashboardScraper import DashboardScraper
from src.arbitrageOptimizer import ArbitrageOptimizer


In [95]:
### get current games
scraper = DashboardScraper()
scraper.connect()
games = scraper.get_json_data()
scraper.disconnect()

In [96]:
### load game into the arbitrageOptimizer
arbitrageOptimize = ArbitrageOptimizer()
for game in games:
    arbitrageOptimize.load_game(game)
    print arbitrageOptimize.get_optimal_ratio()
    print arbitrageOptimize.get_arbitrage_opportunity()

0.0784097651945
None


In [97]:
games

[{'game': {'best_bookie': 'Paddy Power',
   'best_odds': 2.88,
   'date': '2018-02-03 16:00:00',
   'league': 'Northern Ireland: Irish Cup',
   'match_title': 'Ballyclare vs. Glentoran',
   'mean': 2.51,
   'median': 2.88,
   'result_to_bet': 'X',
   'sport': 'soccer',
   'time_to_match': '00:04:15',
   'timestamp': '2018-02-03 16:04:14'},
  'odds_matrix': []}]

# Filter columns by available websites

In [2]:
### imports
import joblib
import pandas as pd
import sys

sys.path.append('..')
from src.arbitrageOptimizer import ArbitrageOptimizer

In [6]:
games = joblib.load('../data/sample.dat')
arbitrageOptimizer = ArbitrageOptimizer()

In [9]:
game = games[0]
arbitrageOptimizer.load_game(game)


In [21]:
available_bookies = ['bet365', 'Interwetten', 'William Hill', 'Unibet', 'bwin', 'Tipico']
bookies_mask = [bookie in available_bookies for bookie in arbitrageOptimizer._odds_matrix.columns]

filtered_odd_matrix = arbitrageOptimizer._odds_matrix.iloc[:,bookies_mask]

In [22]:
filtered_odd_matrix

,bet365,Interwetten,Tipico,Unibet,William Hill
1,1.9,1.95,2.55,1.80,2.10
X,3.6,3.60,3.20,3.45,3.40
2,3.2,3.15,2.55,3.35,2.88


In [19]:
for game in games:
    arbitrageOptimizer.load_game(game)
    print arbitrageOptimizer._odds_matrix.columns

Index([u'10Bet', u'12BET', u'188BET', u'1xBet', u'888sport', u'bet-at-home',
       u'bet365', u'Betclic', u'Betfair', u'Betsafe', u'Betsson', u'BetVictor',
       u'Betway', u'bwin', u'ComeOn', u'Expekt', u'Interwetten', u'mybet',
       u'Paddy Power', u'Pinnacle', u'SBOBET', u'Sportingbet', u'Tipico',
       u'Unibet', u'William Hill', u'youwin'],
      dtype='object')
Index([u'10Bet', u'12BET', u'188BET', u'1xBet', u'888sport', u'bet-at-home',
       u'bet365', u'Betclic', u'Betfair', u'Betsafe', u'Betsson', u'BetVictor',
       u'Betway', u'bwin', u'ComeOn', u'Expekt', u'Interwetten', u'mybet',
       u'Paddy Power', u'Pinnacle', u'SBOBET', u'Sportingbet', u'Tipico',
       u'Unibet', u'William Hill', u'youwin'],
      dtype='object')
Index([u'10Bet', u'12BET', u'188BET', u'1xBet', u'888sport', u'bet-at-home',
       u'bet365', u'Betclic', u'Betfair', u'Betsafe', u'Betsson', u'BetVictor',
       u'Betway', u'bwin', u'ComeOn', u'Expekt', u'Interwetten', u'mybet',
       u'Paddy Pow